In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import networkx.algorithms.isomorphism as iso
import sympy
import numpy as np
import random
import time
import itertools
import math
from IPython.display import clear_output

In [2]:
def graphlet_list(N):
    assert N > 0
    foo = 1
    loc_graphlet_list = {n: [] for n in range(1,N+1)}
    while True:
        G = nx.graph_atlas(foo)
        n = G.number_of_nodes()
        if n>N:
            break
        if nx.is_connected(G):
            loc_graphlet_list[n].append(G)
        foo += 1
    return loc_graphlet_list
    

def find_type_match(T):
    n = T.number_of_nodes()
    if n==1:
        return((0, {u: 0 for u in T.nodes()}))
    if n==2:
        return((0, {u: i for i,u in enumerate(T.nodes())}))
    if n==3:
        if T.number_of_edges()==2:
            u0 = next((node for node in T.nodes() if T.degree(node)==2))
            (u1,u2) = (node for node in T.neighbors(u0))
            return((0, {u0: 0, u1: 1, u2: 2}))
        if T.number_of_edges()==3:
            return((1,{u:i for i,u in enumerate(T.nodes())}))
    if n==4:
        e_num = T.number_of_edges()
        max_degree = max((T.degree(node) for node in T.nodes()))
        if e_num==3 and max_degree==3:
            u3 = next((node for node in T.nodes() if T.degree(node)==3))
            (u0,u1,u2) = (node for node in T.neighbors(u3))
            return((0, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==3 and max_degree==2:
            (u0,u1) = (node for node in T.nodes() if T.degree(node)==2)
            u2 = next((node for node in T.neighbors(u1) if node!=u0))
            u3 = next((node for node in T.neighbors(u0) if node!=u1))
            return((1, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==4 and max_degree==3:
            u3 = next((node for node in T.nodes() if T.degree(node)==3))
            (u1,u2) = (node for node in T.nodes() if T.degree(node)==2)
            u0 = next((node for node in T.nodes() if T.degree(node)==1))
            return((2, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==4 and max_degree==2:
            u0 = next((node for node in T.nodes()))
            (u1,u3) = (node for node in T.neighbors(u0))
            u2 = next((node for node in T.neighbors(u1) if node!=u0))
            return((3, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==5:
            (u0,u2) = (node for node in T.nodes() if T.degree(node)==3)
            (u1,u3) = (node for node in T.nodes() if T.degree(node)==2)
            return((4, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==6:
            (u0,u1,u2,u3) = (node for node in T.nodes())
            return((5, {u0:0, u1:1, u2:2, u3:3}))
    # Improve matching procedure here for n>4.
    GM = next((i, iso.GraphMatcher(T,T_)) 
              for (i,T_) in enumerate(cached_graphlet_list[n]) 
              if iso.GraphMatcher(T,T_).is_isomorphic())
    assert GM[1].is_isomorphic()
    return((GM[0],GM[1].mapping))

def find_type(T):
    n = T.number_of_nodes()
    if n==1:
        return 0
    if n==2:
        return 0
    if n==3:
        if T.number_of_edges()==2:
            return 0
        if T.number_of_edges()==3:
            return 1
    if n==4:
        e_num = T.number_of_edges()
        max_degree = max((T.degree(node) for node in T.nodes()))
        if e_num==3 and max_degree==3:
            return 0
        if e_num==3 and max_degree==2:
            return 1
        if e_num==4 and max_degree==3:
            return 2
        if e_num==4 and max_degree==2:
            return 3
        if e_num==5:
            return 4
        if e_num==6:
            return 5
    # Improve matching procedure here at least for n=4.
    GM = next((i 
              for (i,T_) in enumerate(cached_graphlet_list[n]) 
              if iso.GraphMatcher(T,T_).is_isomorphic()))
    return GM  

def subgraph(G, nodes):
    list_nodes = list(nodes)
    T = nx.Graph()
    T.add_nodes_from(nodes)
    for i in range(len(nodes)):
        for j in range(i):
            if list_nodes[i] in G.neighbors(list_nodes[j]):
                T.add_edge(list_nodes[i],list_nodes[j])
    return T

def sub_edge_num(k, T_type):
    if k < 3:
        return k
    T = cached_graphlet_list[k][T_type]
    count = 0
    for u in T.nodes():
        S = subgraph(T, T.nodes()-{u})
        if not nx.is_connected(S):
            continue
        for v in S.nodes():
            if not nx.is_connected(subgraph(S, S.nodes()-{u})):
                continue
            if not nx.is_connected(subgraph(T, T.nodes()-{v})):
                continue
            count+=1
    return count

def lift(G, vert, k):
    graphlet = set([vert])
    if k==1:
        return graphlet
    u = vert
    neig_list = []
    for n in range(2, k+1):
        neig_list = ([v for v in neig_list if v!=u] 
                     + [v for v in G.neighbors(u) if v not in graphlet])
        u = random.choice(neig_list)
        graphlet.add(u)
    return graphlet

In [3]:
k=4
cached_graphlet_list = graphlet_list(k)
cached_sub_edge_num = {T_type: sub_edge_num(k, T_type) 
                       for T_type in range(len(cached_graphlet_list[k])) 
                      } 


In [10]:
def SRW_step(G, graphlet):
    del_ins_list = []
    for u in graphlet:
        if not nx.is_connected(subgraph(G, graphlet-{u})):
            continue
        neigh = set()
        for v in graphlet-{u}:
            neigh.update(set(G.neighbors(v)))
        neigh = neigh - graphlet - {u}
        for w in neigh:
            del_ins_list.append((u,w))
    pair = random.choice(del_ins_list)
    new_graphlet = graphlet-{pair[0]}; new_graphlet.add(pair[1])
    return (new_graphlet, len(del_ins_list))


def load_graph(name, N=3):
    
    ground_truth = None
    G = None

    if name=='com-amazon':
        G = nx.read_edgelist(
            'Graphs/com-amazon.ungraph.txt',
            create_using = nx.Graph())
        
        if N==3:
            ground_truth = {0: 7750799, 
                            1: 667129}
        if N==4:
            ground_truth = {0: 124295537, 
                            1: 37383434, 
                            2: 13674662, 
                            3: 422515, 
                            4: 1874925, 
                            5: 275961}

    if name=='com-dblp':
        G = nx.read_edgelist(
            'Graphs/com-dblp.ungraph.txt',
            create_using = nx.Graph())
        if N==3:
            ground_truth = {0: 15107734, 
                            1: 2224385}
        if N==4:
            ground_truth = {0: 258570802, 
                            1: 252447350, 
                            2: 96615211, 
                            3: 203394, 
                            4: 4764685, 
                            5: 16713192}

    if name=='com-lj':
        G = nx.read_edgelist(
            'Graphs/com-lj.ungraph.txt',
            create_using = nx.Graph())
        if N==3:
            ground_truth = {0: 3722307805, 
                            1: 177820130}
        if N==4:
            ground_truth = {0: 1983908933796,
                            1: 542683013686,
                            2: 57662704306,
                            3: 2541452010,
                            4: 8190586835,
                            5: 521691844}

    if name=='com-youtube':
        G = nx.read_edgelist(
            'Graphs/com-youtube.ungraph.txt',
            create_using = nx.Graph())
        if N==3:
            ground_truth = {0: 1465313402, 
                            1: 3056386}
        if N==4:
            ground_truth = {0: 5730407268993,
                            1: 91488735459,
                            2: 12371157628,
                            3: 231979854,
                            4: 221833272,
                            5: 4986965}

    if name=='misc-net25':
        G = nx.read_edgelist(
            'Graphs/misc-net25.mtx',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if N==3:
            ground_truth = {0: 12690840, 
                            1: 64090}
        if N==4:
            ground_truth = {0: 361490550,
                            1: 550792350,
                            2: 12554670,
                            3: 44915955,
                            4: 0,
                            5: 0}

    if name=='bio-celegansneural':
        G = nx.read_edgelist(
            'Graphs/bio-celegansneural.mtx',
            create_using = nx.Graph(), data=(('weight',float),))

    if name=='bio-yeast':
        G = nx.read_edgelist(
            'Graphs/bio-yeast.mtx',
            create_using = nx.Graph())
    
    if name=='bn-macaque-rhesus_brain_1':
        G = nx.read_edgelist(
            'Graphs/bn-macaque-rhesus_brain_1.edges',
            create_using = nx.Graph())
    
    if name=='bn-mouse_brain_1':
        G = nx.read_edgelist(
            'Graphs/bn-mouse_brain_1.edges',
            create_using = nx.Graph())
    
    if name=='ia-email-univ':
        G = nx.read_edgelist(
            'Graphs/ia-email-univ.mtx',
            create_using = nx.Graph())

    if name=='misc-polblogs':
        G = nx.read_edgelist(
            'Graphs/misc-polblogs.mtx',
            create_using = nx.Graph(), data=(('weight',float),))
        

    if name=='misc-as-caida':
        G = nx.read_edgelist(
            'Graphs/misc-as-caida.mtx',
            create_using = nx.Graph(), data=(('weight',float),)) 
        if N==3:
            ground_truth = {0: 59513652, 
                            1: 72730}
        if N==4:
            ground_truth = {0: 62565214368,
                            1: 2808802860,
                            2: 203097552,
                            3: 3774144,
                            4: 4084544,
                            5: 0}

    if name=='misc-fullb':
        G = nx.read_edgelist(
            'Graphs/misc-fullb.mtx',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if N==3:
            ground_truth = {0: 162067420, 
                            1: 60212260}
        if N==4:
            ground_truth = {0: 1078734774,
                            1: 4837795036,
                            2: 2707584768,
                            3: 64898820,
                            4: 897215295,
                            5: 370980150}

    if name=='misc-neos3':
        G = nx.read_edgelist(
            'Graphs/misc-neos3.mtx',
            create_using = nx.Graph(), data=(('weight',float),))

        if N==3:
            ground_truth = {0: 207426691, 
                            1: 505603}
        if N==4:
            ground_truth = {0: 59618248397,
                            1: 11164704825,
                            2: 120388385,
                            3: 2047846,
                            4: 499122,
                            5: 0}

    if name=='misc-discogs_affiliation':
        G = nx.read_edgelist(
            'Graphs/misc-discogs_affiliation.edges',
            create_using = nx.Graph())
        if N==3:
            ground_truth = None
        if N==4:
            ground_truth = {0: 208345722513295,
                            1: 851118877585,
                            2: 58223406336,
                            3: 3008868833,
                            4: 439215089,
                            5: 654413}

    if name=='misc-amazon-ratings':
        G = nx.read_edgelist(
            'Graphs/misc-amazon-ratings.edges',
            create_using = nx.Graph())
        if N==3:
            ground_truth = {0: 699425719, 
                            1: 79638}
        if N==4:
            ground_truth = {0: 719668204837,
                            1: 40966346985,
                            2: 184396006,
                            3: 37045086,
                            4: 561566,
                            5: 671}

    if name=='misc-dbpedia-all':
        G = nx.read_edgelist(
            'Graphs/misc-dbpedia-all.edges',
            create_using = nx.Graph())
        if N==3:
            ground_truth = {0: 174250340949, 
                            1: 8329548}
        if N==4:
            ground_truth = {0: 19646604300441472,
                            1: 1652259549599,
                            2: 622928133900,
                            3: 15925209557,
                            4: 15630164176,
                            5: 4609834}
            
    if G is None:
        raise KeyError

    return {'graph': G, 'ground_truth': ground_truth}

def psrw_mixing_variance(G, k, steps_num=1000, burn_in_limit=20):
    v = random.choice(list(G.nodes()))
    init_graphlet = lift(G, v, k-1)
    old_graphlet = init_graphlet
    graphlet_num = len(cached_graphlet_list[k])
    exp_counter = {i:0 for i in range(graphlet_num)}
    var_counter = {i:0 for i in range(graphlet_num)}
    type_counter = {i:0 for i in range(graphlet_num)}
    pair_counter = {i: 
                    {burn_in: 0 
                     for burn_in in range(0,burn_in_limit)}
                    for i in range(graphlet_num)} 

    memory = [None for _ in range(burn_in_limit)]
    for _ in range(steps_num):
        new_graphlet = SRW_step(G, old_graphlet)[0]
        T = old_graphlet.union(new_graphlet)
        old_graphlet = new_graphlet
        assert len(T)==k
        T_type = find_type(subgraph(G, T))
        T_prob = cached_sub_edge_num[T_type]
        type_counter[T_type] += 1
        exp_counter[T_type] += (T_prob)**(-1)
        var_counter[T_type] += (T_prob)**(-2)
        ind = 0
        while ind < burn_in_limit and memory[ind] is not None:
            S_type = memory[ind]
            if T_type==S_type:
                pair_counter[T_type][ind] += 1
            ind+=1
        memory = [T_type] + memory[:-1]

    beta_coeff = {i: [abs(pair_counter[i][burn_in]
                          *(steps_num-burn_in)
                          *type_counter[i]**(-2) - 1)
                      for burn_in in range(burn_in_limit)] 
                  for i in range(graphlet_num) 
                  if type_counter[i]!=0}

    variance = {i: (var_counter[i]*steps_num
                    *exp_counter[i]**(-2))
                for i in range(graphlet_num) 
                if exp_counter[i]!=0}
    
    for i in range(graphlet_num):
        print ("Graphlet ID{}".format(i))
        print("Variance")
        print(variance[i])
        print("Count")
        print(type_counter[i])
        print(" ")
        
    for i in range(graphlet_num):
        print ("Graphlet ID{}".format(i))
        for burn_in in range(burn_in_limit):
            print(beta_coeff[i][burn_in])
        
    return (beta_coeff, variance)

def psrw_count(G, k, steps_num=1000, burn_in=10):
    v = random.choice(list(G.nodes()))
    init_graphlet = lift(G, v, k-1)
    old_graphlet = init_graphlet
    graphlet_num = len(cached_graphlet_list[k])
    exp_counter = {i:0 for i in range(graphlet_num)}

    for _ in range(steps_num):
        new_graphlet = SRW_step(G, old_graphlet)[0]
        T = old_graphlet.union(new_graphlet)
        old_graphlet = new_graphlet
        assert len(T)==k
        T_type = find_type(subgraph(G, T))
        T_prob = cached_sub_edge_num[T_type]
        exp_counter[T_type] += (T_prob)**(-1)

    exp_counter = {i: exp_counter[i]*(steps_num)**(-1)
                   for i in range(graphlet_num)}

    return exp_counter

In [5]:
G = load_graph('misc-neos3',4)['graph']
print(G.number_of_nodes(), G.number_of_edges())

(518832, 2055024)


In [11]:
foo=psrw_count(G,k, steps_num=10**4, burn_in=10)

KeyboardInterrupt: 

In [12]:
#Garbage code

def fullPSRW(G, N, time_limit=None, time_step=10, query_limit=None, 
             epoch_num=1, ground_truth=None):

    assert (time_limit is None) != (query_limit is None)
    norm_error = 0

    for epoch in range(epoch_num):
        init_graphlet = lift(G, random.choice(list(G.nodes())), N-1)
        old_graphlet = init_graphlet
        type_counter = {i:0 for i in range(len(cached_graphlet_list[N]))}
        t0 = time.time()
        iter_count = 0
        query_count = 0
        time_iter_count = 1
        stop_condition = False
        #print('Starting epoch {}'.format(epoch+1))
        type_list = []
        
        while not stop_condition:
            new_graphlet = SRW_step(G, old_graphlet)[0]
            T = old_graphlet.union(new_graphlet)
            old_graphlet = new_graphlet
            assert len(T)==N
            T_type = find_type(subgraph(G, T))
            type_counter[T_type] += (cached_sub_edge_num[T_type])**(-1)
            
            type_list.append(T_type)
            iter_count += 1
            curr_time = time.time()

            if curr_time - t0 > time_iter_count*time_step:
                print("Time is {} NMSE is {}"
                      .format(int(curr_time-t0), 
                              NMSE(type_counter, ground_truth)))
                print("Number of iterations is {}".format(iter_count))
                time_iter_count += 1

            if time_limit is not None:
                stop_condition = (time.time()-t0 > time_limit)
            if query_limit is not None:
                stop_condition = (iter_count > query_limit)
        
        print(type_list)

        if ground_truth is not None:
            error = NMSE(type_counter, ground_truth)
            print("NMSE error is {}".format(error))
            norm_error += error
            
    norm_error = norm_error*(epoch_num)**(-1)
    return {'ratio': normalize(type_counter), 
            'NMSE': norm_error,
            'type_list': type_list
           }

def fullSRW(G, running_time=120, time_step=10, N=3):
    t0 = time.time()
    v0 = next(iter(G.nodes()))
    old_graphlet = set(list(G.neighbors(v0))[:(N-1)]+[v0])
    type_counter = {i:0 for i in range(len(cached_graphlet_list[N]))}
    time_iter_count = 1
    iter_count = 0
    curr_time = time.time()
    while curr_time - t0 < running_time:
        iter_count += 1
        (new_graphlet, old_graphlet_degree) = SRW_step(G, old_graphlet)
        type_counter[find_type(subgraph(G, old_graphlet))] += old_graphlet_degree**(-1)
        old_graphlet = new_graphlet
        curr_time = time.time()
        if curr_time - t0 > time_iter_count*time_step:
            total_count = sum((val for i,val in type_counter.items()))
            print("Time is {} Type counter is {}"
                  .format(int(curr_time-t0),
                          {i: val/total_count for i,val in type_counter.items()}))
            print("Number of iterations is {}".format(iter_count))
            time_iter_count += 1
    total_count = sum((val for i,val in type_counter.items()))
    #print("Time spent {} sec".format(time.time()-t0))
    return {i: val/total_count for i,val in type_counter.items()}

def brute_force(G, N=3):
    type_counter = {i:0 for i in range(len(cached_graphlet_list[N]))}
    if N==3:
        percent_count = 0
        counter = 0
        for u,v in G.edges():
            for w in set(G.neighbors(u))-{v}:
                if w in G.neighbors(v): 
                    type_counter[1] += 1
                else:
                    type_counter[0] += 1
            for w in set(G.neighbors(v))-{u}:
                if w in G.neighbors(u): 
                    type_counter[1] += 1
                else:
                    type_counter[0] += 1
            counter += 1
            if counter > percent_count*0.00001*G.number_of_edges():
                clear_output()
                #print("{}% complete".format(percent_count*0.001))
                percent_count += 1
        type_counter[0] = type_counter[0]/2
        type_counter[1] = type_counter[1]/6
        
    if N==4:
        for u,v in G.edges():
            neigh = set(G.neighbors(u)).union(set(G.neighbors(v)))-{u,v}
            for w,z in itertools.combinations(neigh, 2):
                T = subgraph(G, {u,v,w,z})
                type_counter[find_type(T)] += 1
        type_counter[0] = type_counter[0]/3
        type_counter[1] = type_counter[1]
        type_counter[2] = type_counter[2]/3
        type_counter[3] = type_counter[3]/4
        type_counter[4] = type_counter[4]/5
        type_counter[5] = type_counter[5]/6
    return type_counter

def NMSE(dict_hat, dict_true):
    norm_dict_hat = normalize(dict_hat)
    norm_dict_true = normalize(dict_true)
    return sum(((norm_dict_hat[i]*freq**(-1) - 1)**2
                for i, freq in norm_dict_true.items() if freq != 0))

def normalize(dict_hat):
    total_count = sum((val for i,val in dict_hat.items()))
    return {i: val*(total_count)**(-1) for i,val in dict_hat.items()}

def scale(dict_hat, scalar):
    return {i: int(val*scalar) for i,val in dict_hat.items()}
